In [1]:
import torch
from phi.torch.flow import *

device = torch.device('cuda')

In [2]:
BOUNDS = Box(x=1, y=1, z=1)
RESOLUTION = 100

In [3]:
ORIGIN_DENSITY_FIELD = CenteredGrid(
    values=Noise(),
    extrapolation=extrapolation.ZERO_GRADIENT,
    bounds=BOUNDS,
    resolution=spatial(x=RESOLUTION, y=RESOLUTION, z=RESOLUTION),
)
TARGET_DENSITY_FIELD = CenteredGrid(
    values=Noise(),
    extrapolation=extrapolation.ZERO_GRADIENT,
    bounds=BOUNDS,
    resolution=spatial(x=RESOLUTION, y=RESOLUTION, z=RESOLUTION),
)
ORIGIN_VELOCITY_FIELD = StaggeredGrid(
    values=Noise(),
    extrapolation=extrapolation.ZERO,
    bounds=BOUNDS,
    resolution=spatial(x=RESOLUTION, y=RESOLUTION, z=RESOLUTION),
)

In [4]:
dir_torch = torch.randn(3, device=device, requires_grad=True)
dir_phi = wrap(dir_torch, channel(vector='x,y,z'))
FORCE_FIELD = StaggeredGrid(
    values=dir_phi,
    extrapolation=extrapolation.ZERO,
    bounds=BOUNDS,
    resolution=spatial(x=RESOLUTION, y=RESOLUTION, z=RESOLUTION),
)
OPTIMIZER = torch.optim.RAdam([dir_torch], lr=0.001)

DENSITY = ORIGIN_DENSITY_FIELD
VELOCITY = ORIGIN_VELOCITY_FIELD
dt = 0.2
for _ in range(100):
    OPTIMIZER.zero_grad()

    VELOCITY = VELOCITY + FORCE_FIELD
    DENSITY = advect.mac_cormack(DENSITY, VELOCITY, dt)
    loss = torch.nn.functional.mse_loss(DENSITY.values.native('x,y,z'), TARGET_DENSITY_FIELD.values.native('x,y,z'))
    loss.backward()

    OPTIMIZER.step()

RuntimeError: Output 0 of UnbindBackward0 is a view and its base or another view of its base has been modified inplace. This view is the output of a function that returns multiple views. Such functions do not allow the output views to be modified inplace. You should replace the inplace operation by an out-of-place one.